In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
races_df = spark.read.parquet(f"{processed_folder_path}/races") \
    .withColumnRenamed("name", "race_name") \
    .withColumnRenamed("race_timestamp", "race_date") 



    

In [0]:
circuits_df = spark.read.parquet(f"{processed_folder_path}/circuits") \
    .withColumnRenamed("location", "circuit_location") 


In [0]:
drivers_df = spark.read.parquet(f"{processed_folder_path}/drivers") \
    .withColumnRenamed("name", "driver_name") \
    .withColumnRenamed("number", "driver_number") \
    .withColumnRenamed("nationality", "driver_nationality") \



In [0]:
constructors_df = spark.read.parquet(f"{processed_folder_path}/constructors") \
    .withColumnRenamed("name", "team")

In [0]:
results_df = spark.read.parquet(f"{processed_folder_path}/results") \
    .filter(f"file_date = '{v_file_date}'") \
    .withColumnRenamed("time", "race_time") \
    .withColumnRenamed("race_id", "result_race_id") \
    .withColumnRenamed("file_date", "result_file_date") 



In [0]:
race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner") \
.select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location) 


In [0]:
race_results_df = results_df.join(race_circuits_df, results_df.result_race_id == race_circuits_df.race_id, "inner") \
    .join(drivers_df, results_df.driver_id == drivers_df.driver_id, "inner") \
    .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id, "inner") 


In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_results_df.select("race_id", "race_year", "race_name", "race_date", "circuit_location", 
                                  "driver_name", "driver_number", "team", "nationality", "grid", "fastest_lap", 
                                  "race_time", "points", "position", "result_file_date") \
                                  .withColumn("created_date", current_timestamp()) \
                                  .withColumnRenamed("result_file_date", "file_date")



In [0]:
%sql
SELECT * FROM f1_presentation.race_results WHERE race_name == 'Abu Dhabi Grand Prix' AND race_year == 2020 ORDER BY points desc


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/race_results")

In [0]:
overwrite_partision(final_df, "f1_presentation", "race_results", "race_id")

In [0]:
# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.race_results")

In [0]:
# display(spark.read.parquet(f"{presentation_folder_path}/race_results"))

In [0]:
%sql
SELECT * FROM f1_presentation.race_results WHERE race_year = 2020;

In [0]:
%sql
-- DROP TABLE f1_presentation.race_results;